# Overview

This notebook processes and visualizes the recreation data for the San Gabriel Mountains National Monument.

# Setup

In [9]:
# Import libraries
import arcpy, pandas as pd, numpy as np, arcgis, os
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [10]:
# Set up workspace
arcpy.env.workspace  = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis\sg_output.gdb"
arcpy.env.overwriteOutput = True

In [11]:
# Set gdb path
gdb_path = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis\sg_output.gdb"

# Trails

In [67]:
# Create pandas dataframe from feature class
trails_df = pd.DataFrame.spatial.from_featureclass("sg_trails")

In [68]:
# View the dataframe
trails_df.head()

,OBJECTID,Entity,Handle,Layer,LyrColor,LyrLnType,LyrLineWt,Color,Linetype,Elevation,LineWt,RefName,TRAIL_NO,TRAIL_NAME,TRAIL_TYPE,Length_Miles,Data_Source,Weblink,Trail_Use,Length_Miles_Calc,SHAPE
0,1,3DPolyline,2BBA,Trails,13,Continuous,-3,13,Continuous,0.00,25,,5367848,Marshall Canyon Trail - Alternate parking entr...,Singletrack,0.502409,MTB Project,https://www.mtbproject.com/trail/5367848/marsh...,Hiking | Mountain Biking,0.502409,"{""hasZ"": true, ""paths"": [[[207864.02299999818,..."
1,2,3DPolyline,92,Trails,13,Continuous,-3,13,Continuous,0.00,25,,5220588,Middle Sam Merrill Trail,Singletrack,2.404354,MTB Project,https://www.mtbproject.com/trail/5220588/middl...,Hiking | Mountain Biking,2.404354,"{""hasZ"": true, ""paths"": [[[174085.69060000032,..."
2,3,3DPolyline,31C3,Trails,13,Continuous,-3,13,Continuous,197.97,25,,,"Steep, dirt access road",Doubletrack,0.265695,MTB Project,https://www.mtbproject.com/trail/5496635/steep...,Hiking | Mountain Biking,0.328990,"{""hasZ"": true, ""paths"": [[[192463.02490000054,..."
3,4,3DPolyline,3801,Trails,13,Continuous,-3,13,Continuous,252.98,25,,,Skyline Spur - shaded dirt road,Doubletrack,0.383627,Hiking Project,https://www.hikingproject.com/trail/7005087/sk...,Hiking,0.383627,"{""hasZ"": true, ""paths"": [[[186841.5364000015, ..."
4,5,3DPolyline,2B8C,Trails,13,Continuous,-3,13,Continuous,0.00,25,,7021814,Eaton Canyon Trail,Singletrack,0.486425,Hiking Project,https://www.hikingproject.com/trail/7021814/ea...,Hiking,0.488879,"{""hasZ"": true, ""paths"": [[[174682.6387999989, ..."


In [69]:
# Total number of trails - all uses
trails_df.shape[0]

171

In [70]:
# Total miles of trails - all uses
np.sum(trails_df["Length_Miles"])

381.7430448206554

In [72]:
# Get possible uses for trails
trails_df.Trail_Use.unique()

array(['Hiking | Mountain Biking', 'Hiking', 'Hiking | Equestrian'],
      dtype=object)

## Mountain Biking Trails

In [73]:
# Subset to all trails that allow mountain biking
mtb_mask = trails_df.Trail_Use.apply(lambda x: 'Mountain Biking' in x)
mtb_trails = trails_df[mtb_mask]

In [74]:
# Total miles of trails - mtb (multi-use)
np.sum(mtb_trails["Length_Miles"])

183.07249477543388

In [75]:
# Total number of trails - mtb (multi-use)
mtb_trails.shape[0]

95

## Hiking Trails

In [76]:
# Subset to all trails that allow hiking
hike_mask = trails_df.Trail_Use.apply(lambda x: 'Hiking' in x)
hiking_trails = trails_df[hike_mask]

In [77]:
# Total miles of trails - hiking (multi-use)
np.sum(hiking_trails["Length_Miles"])

381.7430448206554

In [78]:
# Total number of trails - hiking (multi-use)
hiking_trails.shape[0]

171

In [79]:
# Subset to trails that are only hiking, no other activities
hike_only_mask = hiking_trails.Trail_Use.apply(lambda x: x == 'Hiking')
hike_only = hiking_trails[hike_only_mask]

In [80]:
# Total miles of trails - hiking (hiking only)
np.sum(hike_only["Length_Miles"])

193.794954128001

In [81]:
# Total number of trails - hiking (hiking only)
hike_only.shape[0]

72

## Equestrian Trails

In [82]:
# Subset to all trails that allow equestrian use
eq_mask = trails_df.Trail_Use.apply(lambda x: 'Equestrian' in x)
eq_trails = trails_df[eq_mask]

In [83]:
# Total miles of trails - equestrian (multi-use)
np.sum(eq_trails["Length_Miles"])

4.875595917220547

In [84]:
# Total number of trails - equestrian (multi-use)
eq_trails.shape[0]

4

# Paddling

## River Access Sites

In [33]:
# Create pandas dataframe from feature class
river_access_df = pd.DataFrame.spatial.from_featureclass("sg_river_access")

In [34]:
# Number of access sites
river_access_df.shape[0]

7

## Waterways

In [85]:
# Create pandas dataframe from feature class
paddle_df = pd.DataFrame.spatial.from_featureclass("sg_whitewater")

In [86]:
# Total number of miles of whitewater paddling
np.sum(paddle_df["Length_Miles"])

10.900379796297953

In [37]:
# Available difficulties
paddle_df["difficulty"].value_counts()

II-III+       2
IV            1
III-IV+(V)    1
I-II          1
Name: difficulty, dtype: int64

# Rock Climbing

In [38]:
# Create pandas dataframe from feature class
climbing_df = pd.DataFrame.spatial.from_featureclass("sg_climbing")

In [39]:
# Total number of climbing sites
len(climbing_df["Climbing_Site"].unique())

172

In [40]:
# Number of different areas
len(climbing_df["Parent_Site"].unique())

25

# Picnic Areas

In [181]:
# Create pandas dataframe from feature class
picnic_df = pd.DataFrame.spatial.from_featureclass("sg_picnic")

In [182]:
# Total number of picnic sites
len(picnic_df["RECAREANAM"].unique())

25

# Camping

In [183]:
# Create pandas dataframe from feature class
camping_df = pd.DataFrame.spatial.from_featureclass("sg_camping")

In [185]:
# Total number of camping sites
len(camping_df["RECAREANAM"].unique())

25

# Impact of Expansion

In [ ]:
# Goal: see which rec sites are in current monument and which are in expansion - calculate #/% increase with expansion

## Current Rec

In [41]:
# Select AOI by attribute: Status = Designated
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"
selection_type = "NEW_SELECTION"
where_clause = "Status = 'Designated'"

aoi_designated = arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

In [42]:
# Trails
arcpy.analysis.Clip("sg_trails", aoi_designated, "designated_trails")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_trails'>

In [43]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "designated_trails",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'designated_trails'>

In [44]:
# Paddling
arcpy.analysis.Clip("sg_whitewater", aoi_designated, "designated_whitewater")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_whitewater'>

In [45]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "designated_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'designated_whitewater'>

In [46]:
in_layer = "sg_river_access"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_river = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(designated_river, "designated_river")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_river'>

In [47]:
# Climbing
in_layer = "sg_climbing"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_climbing = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(designated_climbing, "designated_climbing")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_climbing'>

In [48]:
# Picnicking
in_layer = "sg_picnic"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_picnic = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(designated_picnic, "designated_picnic")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_picnic'>

In [49]:
# Camping
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = "sg_camping"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_designated

designated_camping = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(designated_camping, "designated_camping")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\designated_camping'>

## Expansion Rec

In [50]:
# Select AOI by attribute: Status = Designated
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-attribute.htm
in_layer = r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"
selection_type = "NEW_SELECTION"
where_clause = "Status = 'Proposed'"

aoi_proposed = arcpy.management.SelectLayerByAttribute(in_layer_or_view = in_layer, 
                                        selection_type = selection_type, 
                                        where_clause = where_clause)

In [52]:
# Trails
arcpy.analysis.Clip("sg_trails", aoi_proposed, "proposed_trails")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_trails'>

In [53]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "proposed_trails",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'proposed_trails'>

In [54]:
# Paddling
arcpy.analysis.Clip("sg_whitewater", aoi_proposed, "proposed_whitewater")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_whitewater'>

In [55]:
# Recalculate length field after clipping
arcpy.management.CalculateGeometryAttributes(in_features = "proposed_whitewater",
                                             geometry_property = [["Length_Miles", "LENGTH"]],
                                             length_unit = "MILES_INT")

<Result 'proposed_whitewater'>

In [56]:
in_layer = "sg_river_access"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_river = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_river, "proposed_river")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_river'>

In [57]:
# Climbing
in_layer = "sg_climbing"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_climbing = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_climbing, "proposed_climbing")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_climbing'>

In [58]:
# Picnicking
in_layer = "sg_picnic"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_picnic = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_picnic, "proposed_picnic")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_picnic'>

In [59]:
# Camping
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm
in_layer = "sg_camping"
overlap_type = "INTERSECT"
selection_type = "SUBSET_SELECTION"
selecting_features = aoi_proposed

proposed_camping = arcpy.management.SelectLayerByLocation(in_layer = in_layer, 
                                                       overlap_type = overlap_type, 
                                                       select_features = selecting_features,
                                                       selection_type = selection_type)
arcpy.management.CopyFeatures(proposed_camping, "proposed_camping")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\proposed_camping'>

## Comparison

In [60]:
# Define function to do comparison
def compare(designated, proposed):
    return (proposed / designated) * 100

### Trails

In [61]:
# Trails
designated_trails = pd.DataFrame.spatial.from_featureclass("designated_trails")
proposed_trails = pd.DataFrame.spatial.from_featureclass("proposed_trails")

In [62]:
# Total number of trails
print(designated_trails.shape[0])
print(proposed_trails.shape[0])

62
122


In [63]:
# Total miles of trails
print(np.sum(designated_trails["Length_Miles"]))
print(np.sum(proposed_trails["Length_Miles"]))

217.94571085837902
163.90815454309438


In [64]:
# % increase
compare(np.sum(designated_trails["Length_Miles"]), np.sum(proposed_trails["Length_Miles"]))

75.20595560130192

In [65]:
# Hiking only designated
hike_only_mask = designated_trails.Trail_Use.apply(lambda x: x == 'Hiking')
hike_only_d = designated_trails[hike_only_mask]
print(np.sum(hike_only_d["Length_Miles"]))
print(hike_only_d.shape[0])

# Hiking only proposed
hike_only_mask = proposed_trails.Trail_Use.apply(lambda x: x == 'Hiking')
hike_only_p = proposed_trails[hike_only_mask]
print(np.sum(hike_only_p["Length_Miles"]))
print(hike_only_p.shape[0])

148.70426192513256
36
45.1791247284311
41


In [27]:
# % increase
compare(np.sum(hike_only_d["Length_Miles"]), np.sum(hike_only_p["Length_Miles"]))

30.38186272776477

In [28]:
# MTB designated
mtb_mask = designated_trails.Trail_Use.apply(lambda x: 'Mountain Biking' in x)
mtb_trails_d = designated_trails[mtb_mask]
print(np.sum(mtb_trails_d["Length_Miles"]))
print(mtb_trails_d.shape[0])

# MTB proposed
mtb_mask = proposed_trails.Trail_Use.apply(lambda x: 'Mountain Biking' in x)
mtb_trails_p = proposed_trails[mtb_mask]
print(np.sum(mtb_trails_p["Length_Miles"]))
print(mtb_trails_p.shape[0])

64.36585296647277
22
118.72902997410607
81


In [29]:
# % increase
compare(np.sum(mtb_trails_d["Length_Miles"]), np.sum(mtb_trails_p["Length_Miles"]))

184.45965446298098

In [30]:
# Equestrian proposed
eq_mask = designated_trails.Trail_Use.apply(lambda x: 'Equestrian' in x)
eq_trails_d = designated_trails[eq_mask]
print(np.sum(eq_trails_d["Length_Miles"]))
print(eq_trails_d.shape[0])

# Equestrian proposed
eq_mask = proposed_trails.Trail_Use.apply(lambda x: 'Equestrian' in x)
eq_trails_p = proposed_trails[eq_mask]
print(np.sum(eq_trails_p["Length_Miles"]))
print(eq_trails_p.shape[0])

4.875595917220547
4
0.0
0


In [32]:
# % increase
compare(np.sum(eq_trails_d["Length_Miles"]), np.sum(eq_trails_p["Length_Miles"]))

0.0

### Waterways

In [14]:
# Whitewater paddling
designated_whitewater = pd.DataFrame.spatial.from_featureclass("designated_whitewater")
proposed_whitewater = pd.DataFrame.spatial.from_featureclass("proposed_whitewater")

In [15]:
print(np.sum(designated_whitewater["Length_Miles"]))
print(np.sum(proposed_whitewater["Length_Miles"]))

8.376249732588741
2.524130063709214


In [33]:
# % increase
compare(np.sum(designated_whitewater["Length_Miles"]), np.sum(proposed_whitewater["Length_Miles"]))

30.13436972740679

In [16]:
# River access
designated_river = pd.DataFrame.spatial.from_featureclass("designated_river")
proposed_river = pd.DataFrame.spatial.from_featureclass("proposed_river")

In [17]:
print(designated_river.shape[0])
print(proposed_river.shape[0])

5
2


In [34]:
# % increase
compare(designated_river.shape[0], proposed_river.shape[0])

40.0

### Climbing

In [18]:
# Climbing
designated_climbing = pd.DataFrame.spatial.from_featureclass("designated_climbing")
proposed_climbing = pd.DataFrame.spatial.from_featureclass("proposed_climbing")

In [19]:
print(len(designated_climbing["Climbing_Site"].unique()))
print(len(proposed_climbing["Climbing_Site"].unique()))

167
3


In [35]:
# % increase
compare(len(designated_climbing["Climbing_Site"].unique()), len(proposed_climbing["Climbing_Site"].unique()))

1.7964071856287425

### Picnic

In [37]:
# Picnic
designated_picnic = pd.DataFrame.spatial.from_featureclass("designated_picnic")
proposed_picnic = pd.DataFrame.spatial.from_featureclass("proposed_picnic")

In [38]:
print(len(designated_picnic["RECAREANAM"].unique()))
print(len(proposed_picnic["RECAREANAM"].unique()))

15
10


In [39]:
# % increase
compare(len(designated_picnic["RECAREANAM"].unique()), len(proposed_picnic["RECAREANAM"].unique()))

66.66666666666666

### Camping

In [41]:
# Camping
designated_camping = pd.DataFrame.spatial.from_featureclass("designated_camping")
proposed_camping = pd.DataFrame.spatial.from_featureclass("proposed_camping")

In [42]:
print(len(designated_camping["RECAREANAM"].unique()))
print(len(proposed_camping["RECAREANAM"].unique()))

22
3


In [43]:
# % increase
compare(len(designated_camping["RECAREANAM"].unique()), len(proposed_camping["RECAREANAM"].unique()))

13.636363636363635

# Clustering Analysis

In [194]:
# Get point features for the start/end of trails
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/feature-vertices-to-points.htm
arcpy.management.FeatureVerticesToPoints(in_features = "Trails\Trails",
                                         out_feature_class = "trail_points",
                                         point_location = "BOTH_ENDS")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\trail_points'>

In [212]:
# Merge all the rec points together into one layer
# Define layers
trail_points = os.path.join(gdb_path, "trail_points")
sg_picnic = os.path.join(gdb_path, "sg_picnic")
sg_camping = os.path.join(gdb_path, "sg_camping")
river_access = os.path.join(gdb_path, "River_Access\River_Access")
climbing = os.path.join(gdb_path, "Rock_Climbing\Rock_Climbing_Site")

In [213]:
# Create field mapping object
field_mappings = arcpy.FieldMappings()

# Add input field for rec area name into new output field
map_name = arcpy.FieldMap()
map_name.addInputField(trail_points, "TRAIL_NAME")
map_name.addInputField(climbing, "Parent_Site")
map_name.addInputField(river_access, "Section_Name")
map_name.addInputField(sg_picnic, "RECAREANAM")
map_name.addInputField(sg_camping, "RECAREANAM")

# Set name of new output field for site name
rec_name = map_name.outputField
rec_name.name = "Rec_Name"
map_name.outputField = rec_name

# Add output fields to field mappings object
field_mappings.addFieldMap(map_name)

In [214]:
# Merge
arcpy.management.Merge([trail_points, river_access, climbing, sg_picnic, sg_camping], 
                       "all_rec", 
                       field_mappings,
                      add_source = "ADD_SOURCE_INFO")

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\all_rec'>

In [211]:
# Run the Density-Based Clustering analysis
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/densitybasedclustering.htm

arcpy.stats.DensityBasedClustering(in_features = "all_rec", 
                                   output_features = "rec_density", 
                                   cluster_method = "HDBSCAN", # TODO try OPTICS
                                   min_features_cluster = 10)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\rec_density'>

# Maps

## Map all recreation types

In [91]:
# Set up environment
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx.defaultGeodatabase = gdb_path

In [92]:
# all_rec = aprx.createMap(name = "Recreation",
#                         map_type = "MAP")
all_rec = aprx.listMaps("Recreation")[0] 

In [96]:
# Add recreation data
all_rec.addDataFromPath(os.path.join(gdb_path, r"Prop_SG_Monument_Expansion\Proposed_San_Gabriel_National_Monument_Expansion"))
all_rec.addDataFromPath(os.path.join(gdb_path, r"Trails\Trails"))
all_rec.addDataFromPath(os.path.join(gdb_path, r"Whitewater_Paddling\Whitewater_Paddling"))
all_rec.addDataFromPath(os.path.join(gdb_path, r"Rock_Climbing\Rock_Climbing_Site"))
all_rec.addDataFromPath(os.path.join(gdb_path, "sg_picnic"))
all_rec.addDataFromPath(os.path.join(gdb_path, "sg_camping"))

In [98]:
aoi = all_rec.listLayers("Proposed_San_Gabriel_National_Monument_Expansion")[0]
trails = all_rec.listLayers("Trails")[0]
paddle = all_rec.listLayers("Whitewater_Paddling")[0]
picnic = all_rec.listLayers("sg_picnic")[0]
camp = all_rec.listLayers("sg_camping")[0]
climb = all_rec.listLayers("Rock_Climbing_Site")[0]

all_lyr = [trails, paddle, picnic, camp, climb, aoi]

In [104]:
for lyr in all_lyr:
    sym = lyr.symbology
    # Update symbology for recreation types
    if(lyr.name == "Trails"):
        sym.renderer.symbol.color = {"RGB": [0, 0, 0, 100]}
    elif(lyr.name == "Whitewater_Paddling"):
        sym.renderer.symbol.color = {"RGB": [0, 92, 230, 100]}
    elif(lyr.name == "sg_picnic"):
        sym.renderer.symbol.color = {"RGB": [255, 0, 197, 100]}
    elif(lyr.name == "sg_camping"):
        sym.renderer.symbol.color = {"RGB": [58, 45, 168, 100]}
    elif(lyr.name == "Rock_Climbing_Site"):
        sym.renderer.symbol.color = {"RGB": [166, 216, 84, 100]}
    
    # Update symbology for AOI polygons
    elif(lyr.name == "Proposed_San_Gabriel_National_Monument_Expansion"):
        sym.updateRenderer("UniqueValueRenderer")
        sym.renderer.fields = ["Status"]
        sym.renderer.colorRamp = aprx.listColorRamps("Set 2 (3 classes)")[0]
        
    lyr.symbology = sym